# Model Deployment using Numpy Linear Classifier

## 1. Introduction
In this workbook, we will look into the basics of deploying a model. For simplicity, we will consider a simple numpy linear classifier $$ \mathbf{Y} = \mathbf{W} \mathbf{X} + \mathbf{b}$$

For simplicity, we will consider $\mathbf{X}$ to be 6 dimensional ($\mathbb{R}^6$). i.e. 1 data point $x \in \mathbf{X}$ will be a numpy array of shape $(1,6)$. The output $\mathbf{Y}$ is 3 dimensional ($\mathbb{R}^3$). Then, the weights $\mathbf{W}$ will be a numpy array of shape $(3,6)$ and bias $\mathbf{b}$ will be a numpy array of shape $(,3)$. 

In this workbook, we will demonstrate how to deploy this numpy linear classifier as a server and how to perform query on this numpy linear classifier.

## 2. Imports and Dependencies.
The few packages needed are loaded next. Particularly, `numpy`, `mlflow` will be majorly used in this tutorial. `requests` package will be used for performing query. `json` is used to post and get response from the server.

In [1]:
import os
import sys
import mlflow
import numpy as np
from mlflow import pyfunc

# Setting a tracking uri to log the mlflow logs in a particular location tracked by 
from mlflow.tracking import MlflowClient
tracking_uri = os.environ.get("TRACKING_URL")
client = MlflowClient(tracking_uri=tracking_uri)
mlflow.set_tracking_uri(tracking_uri)

# 3. Some utility functions

In [2]:
## Utility function to add libraries to conda environment
def add_libraries_to_conda_env(_conda_env,libraries=[],conda_dependencies=[]):
    dependencies = _conda_env["dependencies"]
    dependencies = dependencies + conda_dependencies
    pip_index = None
    for _index,_element in enumerate(dependencies):
        if type(_element) == dict:
            if "pip" in _element.keys():
                pip_index = _index
                break
    dependencies[pip_index]["pip"] =  dependencies[pip_index]["pip"] + libraries
    _conda_env["dependencies"] = dependencies
    return _conda_env

In [3]:
## Model Wrapper that takes X as input using json and predicts an output Y
class ModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self,context):
        import numpy as np
        self.model = np.load(context.artifacts['model_path'], allow_pickle=True).tolist()
        print("Model initialized")
    
    def predict(self, context, model_input):
        import numpy as np
        import json
        json_txt = ", ".join(model_input.columns)
        data_list = json.loads(json_txt)
        inputs = np.array(data_list)
        if len(inputs.shape) == 2:
            print('batch inference')
            predictions = []
            for idx in range(inputs.shape[0]):
                prediction = np.matmul(inputs[idx,:],self.model['weights'].T) + self.model['bias']
                predictions.append(prediction.tolist())
        elif len(inputs.shape) == 1:
            print('single inference')
            predictions = self.model['weights'].T * inputs + self.model['bias']
            predictions = predictions.tolist()
        else:
            raise ValueError('invalid input shape')
        return json.dumps(predictions)

In [4]:
# instantiate the python inference model wrapper for the server
model_wrapper = ModelWrapper()
env = mlflow.pyfunc.get_default_conda_env()
env = add_libraries_to_conda_env(env,libraries=['numpy'])

# define the model weights randomly
np_weights = np.random.rand(3,6)
np_bias = np.random.rand(3)

# checkpointing and logging the model in mlflow
artifact_path = './np_model'
np.save(artifact_path, {'weights':np_weights, 'bias':np_bias})
model_artifacts = {"model_path" : artifact_path+'.npy'}
mlflow.pyfunc.log_model("np_model", python_model=model_wrapper, artifacts=model_artifacts, conda_env=env)

## 4. Deploying the model
The above code logs a model in the experiments tab. For more info please refer [here](https://rocketml.gitbook.io/rocketml-user-guide/experiments). After deploying the model, we can obtain the model url for performing query as shown below.

## 5. Query from the server

There are two methods to perform query... The first is using `requests` library and the other using `curl` shell command.

In [5]:
import requests
import json

url = "https://oqgq98z.sciml.rocketml.net/invocations"
headers = {"Content-Type":"text/csv"}

# First case, run inference on single data point
np_array = np.random.rand(1,6).tolist()
json_data = json.dumps(np_array)
response = requests.post(url,data=json_data,headers=headers)
if response.status_code == 200:
    output = np.array(json.loads(response.json())).astype(np.float32)
    print(output)
else:
    print(response.status_code)
    print("REST API deployment is in progress -- please try again in a few minutes!")

# Second case, run inference on multiple data points
np_array = np.random.rand(20,6).tolist()
json_data = json.dumps(np_array)
response = requests.post(url,data=json_data,headers=headers)
if response.status_code == 200:
    output = np.array(json.loads(response.json())).astype(np.float32)
    print(output)
else:
    print(response.status_code)
    print("REST API deployment is in progress -- please try again in a few minutes!")

[[3.4653852 2.0398915 3.1586297]]
[[2.5898142 1.8786722 2.5708313]
 [2.5593674 2.0349865 2.6073494]
 [3.6068127 2.184563  3.434891 ]
 [2.902356  2.3069386 2.953126 ]
 [2.637235  2.029517  2.6305063]
 [2.1237707 1.4357289 2.2697406]
 [3.0577552 1.9615693 3.0917268]
 [3.000336  2.0551164 2.9306874]
 [1.7696865 1.1558454 2.0506356]
 [2.4707701 1.7810833 2.9085393]
 [2.7724752 1.9395614 2.7040203]
 [3.3025193 2.3785317 3.3959422]
 [1.283288  0.5862225 1.4935229]
 [2.6822236 1.6249593 2.9523573]
 [2.6495163 1.9702814 2.6044328]
 [2.5158823 2.1459305 2.6397395]
 [2.6616333 1.9748728 2.554915 ]
 [2.0908048 1.3619525 2.3795984]
 [1.4215319 1.0770459 1.7504961]
 [2.0703692 1.4841241 2.2010443]]


In [6]:
!curl https://oqgq98z.sciml.rocketml.net/invocations -H 'Content-Type:text/csv' -d '[[0.6499166977064089, 0.17579454262114602, 0.2688911143313131, 0.7146591854799202, 0.6497433572112488, 0.7723469203958951]]'

/bin/bash: curl: command not found
